In [1]:
  #!/usr/bin/env python

try:
    import gi
    gi.require_version('NumCosmo', '1.0')
    gi.require_version('NumCosmoMath', '1.0')
except:
    pass

import math
import matplotlib.pyplot as plt
from gi.repository import GObject
from gi.repository import NumCosmo as nc
from gi.repository import NumCosmoMath as ncm
import numpy as np
from astropy.io import fits
from astropy.table import Table
import sys
sys.path.insert(0,'../../scripts')

import pyccl as ccl

from nc_ccl import create_nc_obj, ccl_cosmo_set_high_prec

ncm.cfg_init()
ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

initialize the `Cosmological models`

In [2]:
#CCL cosmology
cosmo_ccl = ccl.Cosmology(Omega_c = 0.30711 - 0.048254, Omega_b = 0.048254, h = 0.677, sigma8 = 0.8822714165197718, n_s=0.96, Omega_k = 0, transfer_function='eisenstein_hu')
#ccl_cosmo_set_high_prec (cosmo_ccl)

cosmo_numcosmo, dist, ps_lin, ps_nln, hmfunc = create_nc_obj (cosmo_ccl)

psf = hmfunc.peek_psf ()

# Define proxy modelling
Use a mass proxy, define the probability for observing a proxy given a mass and redhsift

$$
P(\log\lambda|M,z) = N(\mu(M,z), \sigma^2(M,z))
$$
the mean is
$$
\mu(M,z) = \mu_0 + a_\mu^M\log_{10}\frac{M}{M_0} + a_\mu^z\log_{10}\frac{1+z}{1+z_0} 
$$
variance is
$$
\sigma(M,z) = \sigma_0 + a_\sigma^M\log_{10}\frac{M}{M_0} + a_\sigma ^z\log_{10}\frac{1+z}{1+z_0} 
$$

In [3]:
#CosmoSim_proxy model
#M_0, z_0
theta_pivot = [3e14/0.71, 0.6]
#\mu_0, a_\mu^z, a_\mu^M
theta_mu = [3.19, -0.7, 2]
#\sigma_0, a_\sigma^z, a_\sigma^M
theta_sigma = [0.33, 0.,-0.08]
#Richness object

area = (0.25)*4*np.pi / 100.0
lnRl = 1.0
lnRu = 2.0
zl = 0.25
zu = 1.0

#Numcosmo_proxy model
cluster_z = nc.ClusterRedshift.new_from_name("NcClusterRedshiftNodist{'z-min': <%20.15e>, 'z-max':<%20.15e>}" % (zl, zu))
cluster_m = nc.ClusterMass.new_from_name("NcClusterMassAscaso{'M0':<%20.15e>,'z0':<%20.15e>,'lnRichness-min':<%20.15e>, 'lnRichness-max':<%20.15e>}" % (3e14/(0.71),0.6, lnRl, lnRu))
cluster_m.param_set_by_name('mup0', 3.19)
cluster_m.param_set_by_name('mup1', 2/np.log(10))
cluster_m.param_set_by_name('mup2', -0.7/np.log(10))
cluster_m.param_set_by_name('sigmap0', 0.33)
cluster_m.param_set_by_name('sigmap1', -0.08/np.log(10))
cluster_m.param_set_by_name('sigmap2', 0/np.log(10))

initialize the `ClusterAbundance` object

In [4]:
#Numcosmo Cluster Abundance

#First we need to define the multiplicity function here we will use the tinker
mulf = nc.MultiplicityFuncTinker.new()
mulf.set_linear_interp (True)
mulf.set_mdef(nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta(200)
#Second we need to construct a filtered power spectrum 

hmf = nc.HaloMassFunction.new(dist,psf,mulf)
hmf.set_area(area)

ca = nc.ClusterAbundance.new(hmf,None)
mset = ncm.MSet.new_array([cosmo_numcosmo,cluster_m,cluster_z])

In [5]:
ncount = Nc.DataClusterNCount.new (ca, "NcClusterRedshiftNodist", "NcClusterMassAscaso")
ncount.catalog_load ("ncount_ascaso.fits")

In [6]:
cosmo_numcosmo.props.Omegac_fit = True
cosmo_numcosmo.props.w0_fit = True
cluster_m.props.mup0_fit = True
mset.prepare_fparam_map ()

In [7]:
ncount.set_binned (False)

In [8]:
dset = ncm.Dataset.new ()
dset.append_data (ncount)

In [9]:
lh = Ncm.Likelihood (dataset = dset)
fit = Ncm.Fit.new (Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_FORWARD)

In [ ]:
fitmc = Ncm.FitMC.new (fit, Ncm.FitMCResampleType.FROM_MODEL, Ncm.FitRunMsgs.FULL)
fitmc.set_nthreads (3)

fitmc.set_data_file ("ncount_ascaso_mc_unbinned.fits")

fitmc.start_run ()
fitmc.run_lre (50, 1.0e-2)
fitmc.end_run ()


#----------------------------------------------------------------------------------
# NcmFitMC: Starting Monte Carlo...
#----------------------------------------------------------------------------------
# Data used:
#   - Cluster abundance unbinned
#----------------------------------------------------------------------------------
# NcmFitMC: Fiducial model set:
#----------------------------------------------------------------------------------
# Model[03000]:
#   - NcHICosmo : Chevalier-Polarski-Linder parametrization
#----------------------------------------------------------------------------------
# Model parameters
#   -         H0[00]:  67.7                [FIXED]
#   -     Omegac[01]:  0.258856            [FREE]
#   -     Omegak[02]:  0                   [FIXED]
#   -    Tgamma0[03]:  2.725               [FIXED]
#   -         Yp[04]:  0.24                [FIXED]
#   -       ENnu[05]:  3.046               [FIXED]
#   -     Omegab[06]:  0.048254            [FIXED]
#   -         w

#  gradient evaluations [000000]
#  degrees of freedom   [012009]
#  m2lnL     =    -3020.73635605466 (    -3020.7364 )
#  Fit parameters:
#     0.19977088314497    -1.11909663997147     3.49436326949647    
# NcmMSetCatalog: Current mean:  -3050.2       0.23758     -1.0254       3.3307     
# NcmMSetCatalog: Current msd:    34.298       0.013332     0.029061     0.067794   
# NcmMSetCatalog: Current sd:     90.745       0.035273     0.076887     0.17937    
# NcmMSetCatalog: Current var:    8234.6       0.0012442    0.0059116    0.032172   
# NcmMSetCatalog: Current tau:    1            1            1            1          
# Task:NcmFitMC, completed: 4 of 47, elapsed time: 00:03:32.5598
# Task:NcmFitMC, mean time: 00:00:53.1101 +/- 00:00:47.3794
# Task:NcmFitMC, time left: 00:38:03.7354 +/- 00:33:57.3129
# Task:NcmFitMC, current time:        Wed Apr 06 2022, 00:36:36
# Task:NcmFitMC, estimated to end at: Wed Apr 06 2022, 01:14:40 +/- 00:33:57.3129
#  Elapsed time: 00 days, 00:03:38.8

# Task:NcmFitMC, mean time: 00:00:44.2261 +/- 00:00:25.1271
# Task:NcmFitMC, time left: 00:26:32.1400 +/- 00:15:04.5761
# Task:NcmFitMC, current time:        Wed Apr 06 2022, 00:41:10
# Task:NcmFitMC, estimated to end at: Wed Apr 06 2022, 01:07:42 +/- 00:15:04.5761
#  Elapsed time: 00 days, 00:03:47.6898370
#  iteration            [000149]
#  function evaluations [000151]
#  gradient evaluations [000000]
#  degrees of freedom   [012095]
#  m2lnL     =    -3074.45244683226 (    -3074.4524 )
#  Fit parameters:
#     0.2746398654865     -1.04165147515485     3.05838949064453    
# NcmMSetCatalog: Current mean:  -3077.3       0.25948     -1.0262       3.1858     
# NcmMSetCatalog: Current msd:    32.182       0.0092042    0.019279     0.050059   
# NcmMSetCatalog: Current sd:     124.64       0.035648     0.074669     0.19388    
# NcmMSetCatalog: Current var:    15535        0.0012707    0.0055754    0.037589   
# NcmMSetCatalog: Current tau:    1            1            1            1   

# NcmMSetCatalog: Current msd:    25.554       0.0072425    0.016828     0.038337   
# NcmMSetCatalog: Current sd:     119.86       0.03397      0.078932     0.17982    
# NcmMSetCatalog: Current var:    14366        0.001154     0.0062302    0.032334   
# NcmMSetCatalog: Current tau:    1            1            1            1          
# Task:NcmFitMC, completed: 19 of 47, elapsed time: 00:09:55.3135
# Task:NcmFitMC, mean time: 00:00:31.3290 +/- 00:00:15.6012
# Task:NcmFitMC, time left: 00:14:37.2121 +/- 00:07:16.8346
# Task:NcmFitMC, current time:        Wed Apr 06 2022, 00:42:59
# Task:NcmFitMC, estimated to end at: Wed Apr 06 2022, 00:57:36 +/- 00:07:16.8346
#  Elapsed time: 00 days, 00:02:37.8526560
#  iteration            [000248]
#  function evaluations [000250]
#  gradient evaluations [000000]
#  degrees of freedom   [011861]
#  m2lnL     =    -2803.15137335019 (    -2803.1514 )
#  Fit parameters:
#     0.344985020428757   -0.827282398930888    2.92558368737086    
# NcmMSetCa

#  iteration            [000166]
#  function evaluations [000168]
#  gradient evaluations [000000]
#  degrees of freedom   [011933]
#  m2lnL     =     -3029.3045282896 (    -3029.3045 )
#  Fit parameters:
#     0.282760043991491   -0.967385503239157    3.0795344650772     
# NcmMSetCatalog: Current mean:  -3075.1       0.26356     -1.0105       3.1755     
# NcmMSetCatalog: Current msd:    22.421       0.0065055    0.015403     0.032285   
# NcmMSetCatalog: Current sd:     122.81       0.035632     0.084367     0.17683    
# NcmMSetCatalog: Current var:    15082        0.0012697    0.0071178    0.031271   
# NcmMSetCatalog: Current tau:    1            1            1            1          
# Task:NcmFitMC, completed: 27 of 47, elapsed time: 00:12:06.0797
# Task:NcmFitMC, mean time: 00:00:26.8890 +/- 00:00:11.4036
# Task:NcmFitMC, time left: 00:08:57.7804 +/- 00:03:48.0711
# Task:NcmFitMC, current time:        Wed Apr 06 2022, 00:45:09
# Task:NcmFitMC, estimated to end at: Wed Apr 06 20

# NcmMSetCatalog: Current tau:    1            1            1            1          
# Task:NcmFitMC, completed: 34 of 47, elapsed time: 00:13:51.9363
# Task:NcmFitMC, mean time: 00:00:24.4662 +/- 00:00:09.1990
# Task:NcmFitMC, time left: 00:05:18.0604 +/- 00:01:59.5864
# Task:NcmFitMC, current time:        Wed Apr 06 2022, 00:46:55
# Task:NcmFitMC, estimated to end at: Wed Apr 06 2022, 00:52:13 +/- 00:01:59.5864
#  Elapsed time: 00 days, 00:02:32.9858140
#  iteration            [000209]
#  function evaluations [000211]
#  gradient evaluations [000000]
#  degrees of freedom   [011942]
#  m2lnL     =    -3041.28856174584 (    -3041.2886 )
#  Fit parameters:
#     0.290122693103279   -1.03529304807751     2.98037545941645    
# NcmMSetCatalog: Current mean:  -3084.9       0.26        -1.0162       3.1888     
# NcmMSetCatalog: Current msd:    19.506       0.0054345    0.012568     0.027532   
# NcmMSetCatalog: Current sd:     120.24       0.0335       0.077471     0.16972    
# NcmMSetCa